# Tracking by overlaps: example of custom metric

This example illustrates the usage of custom distance metrics, by tracking segmented cells by its overlap.

In [1]:
%pip install -q --upgrade -r requirements.txt


[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade -i https://test.pypi.org/simple/ laptrack

Looking in indexes: https://test.pypi.org/simple/

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Importing packages

In [4]:
import napari
from skimage.measure import regionprops_table
from matplotlib import pyplot as plt
from itertools import product
import numpy as np
import pandas as pd
from laptrack import LapTrack
from laptrack.metric

## Loading data 

Loading segmentation data and show it in the viewer.

Note: this data is generated by cropping `segmentation.npy` in https://github.com/NoneqPhysLivingMatterLab/cell_interaction_gnn .

In [64]:
labels = np.load("overlap_tracking_data/labels.npy")

In [65]:
viewer = napari.Viewer()
viewer.add_labels(labels)

<Labels layer 'labels' at 0x195d87f70>

## Calculating segmentation overlaps 

In [55]:
### summarizing the 

In [67]:
lo=LabelOverlap(labels)

In [71]:
overlap_records = []
for f in range(labels.shape[0]-1):
    print(f)
    l1s = np.unique(labels[f])
    l1s = l1s[l1s!=0]
    l2s = np.unique(labels[f+1])
    l2s = l2s[l2s!=0]
    for l1,l2 in product(l1s,l2s):
        overlap, iou, ratio_1, ratio_2 = lo.calc_overlap(f,l1,f+1,l2)
        overlap_records.append({
            "frame" : f,
            "label1" : l1,
            "label2" : l2,
            "overlap" : overlap, 
            "iou" : iou, 
            "ratio_1" : ratio_1, 
            "ratio_2" : ratio_2,
        })
overlap_df = pd.DataFrame.from_records(overlap_records)
overlap_df = overlap_df.set_index(["frame","label1","label2"])


0
1
2
3


In [75]:
display(overlap_df)

overlap      iou  ratio_1   ratio_2
frame label1 label2                                     
0     1      1             0  0.00000  0.00000  0.000000
             2             0  0.00000  0.00000  0.000000
             3             0  0.00000  0.00000  0.000000
             4             0  0.00000  0.00000  0.000000
             5            88  0.23219  0.52071  0.295302
...                      ...      ...      ...       ...
3     141    156           0  0.00000  0.00000  0.000000
             157           0  0.00000  0.00000  0.000000
             158           0  0.00000  0.00000  0.000000
             159           0  0.00000  0.00000  0.000000
             160           5  0.16129  1.00000  0.161290

[78988 rows x 4 columns]

## Tracking 

Make the coordinate file: this time (frame, label) to get the overlap data from `overlap_df`.

In [87]:
dfs = []
for frame in range(len(labels)):
    df = pd.DataFrame(regionprops_table(
        labels[frame],properties=["label","centroid"]
    ))
    df["frame"] = frame
    dfs.append(df)
coordinate_df = pd.concat(dfs)
display(coordinate_df)

,label,centroid-0,centroid-1,frame
0,1,148.526627,2.278107,0
1,2,108.118367,3.742857,0
2,3,62.957143,2.964286,0
3,4,249.611111,0.222222,0
4,5,5.279279,3.927928,0
...,...,...,...,...
155,156,130.508982,251.437126,4
156,157,253.722222,253.222222,4
157,158,148.350000,253.500000,4
158,159,180.892857,254.107143,4


In [88]:
def metric(c1,c2):
    (frame1, label1), (frame2, label2) = c1, c2
    if frame1 == frame2 + 1:
        tmp = (frame1, label1)
        (frame1, label1) = (frame2, label2)
        (frame2, label2) = tmp
    assert frame1 + 1 == frame2
    ratio_2 = overlap_df.loc[(frame1, label1, label2)]["ratio_2"]
    return (1-ratio_2)

lt = LapTrack(
    track_dist_metric=metric,
    track_cost_cutoff=0.9,
    splitting_dist_metric=metric,
    splitting_cost_cutoff=0.9,
)

In [89]:
track_df, split_df, _ = lt.predict_dataframe(
    coordinate_df,
    coordinate_cols=["frame","label"],
    only_coordinate_cols=False
)

In [92]:
track_df = track_df.reset_index()
display(track_df)

,level_0,frame,index,label,centroid-0,centroid-1,frame_y,tree_id,track_id
0,0,0,0,1,148.526627,2.278107,0,0,0
1,1,0,1,2,108.118367,3.742857,0,1,1
2,2,0,2,3,62.957143,2.964286,0,2,2
3,3,0,3,4,249.611111,0.222222,0,3,3
4,4,0,4,5,5.279279,3.927928,0,4,4
...,...,...,...,...,...,...,...,...,...
703,703,4,155,156,130.508982,251.437126,4,116,179
704,704,4,156,157,253.722222,253.222222,4,122,122
705,705,4,157,158,148.350000,253.500000,4,129,202
706,706,4,158,159,180.892857,254.107143,4,123,201


In [93]:
display(split_df)

,parent_track_id,child_track_id
0,1,133
1,1,132
2,11,134
3,11,137
4,14,138
...,...,...
107,109,237
108,118,240
109,118,241
110,124,242


In [100]:
viewer.add_tracks(track_df[["track_id","frame","centroid-0","centroid-1"]].values,
                      graph={
        row["child_track_id"]: row["parent_track_id"] for _, row in split_df.iterrows()
    },tail_length=1)

<Tracks layer 'Tracks [2]' at 0x19dc48130>

In [97]:
new_labels = np.zeros_like(labels)
for tree_id, grp in track_df.groupby("tree_id"):
    for _, row in grp.iterrows():
        frame = int(row["frame"])
        label = int(row["label"])
        new_labels[frame][labels[frame]==label] = tree_id+1

In [98]:
viewer.add_labels(new_labels)

<Labels layer 'new_labels' at 0x19daef3d0>